In [1]:
using Distributed
N = 5
addprocs(N*3)

15-element Array{Int64,1}:
  2
  3
  4
  5
  6
  7
  8
  9
 10
 11
 12
 13
 14
 15
 16

In [2]:
nprocs()

16

In [3]:
using PyCall
cma = pyimport("cma")

PyObject <module 'cma' from '/home/mkoehler/.local/lib/python3.8/site-packages/cma/__init__.py'>

In [5]:
@everywhere using DrWatson
@everywhere @quickactivate :Catalyst
using CSV

In [6]:
@everywhere function eval_model(input_args::Array,w::Float64, input_exp::Array, output_exp::Array, micromesh) 
    try 
        return Catalyst.solve(input_args[1], input_args[2], input_args[3], 
                    input_exp, output_exp, progress=false, 
                    microcomp_type=:nonlinear, 
                    Q=input_args[4], kₙ=input_args[5], 
                    calibration=true, w=w, micromesh=micromesh)
    catch err #in case we loose newton convergence in a microscopic computation 
        return 1e10
    end
end

function calibrate(cma,N)
    w = [1.5302e-4*(1/0.37), 1.9128e-4*(1/0.37), 2.2954e-4*(1/0.37)] # Q8, Q10, Q12
    Init = [0.006536800306, 1., 1., 1., 1.] #initial guess
    opts = cma.CMAOptions()
    opts["bounds"] = [[0, 0, 0, 0, 0], [1, 50, 50, 20, 20]] 
    opts["popsize"] = N
    weights = cma.recombination_weights.RecombinationWeights(N)
    opts["CMA_recombination_weights"] = weights
    es = cma.CMAEvolutionStrategy(Init, 0.5, opts) 
    
    #Step 1
    #Read in all 3 different velocity curves
    input_exp_12 = []
    output_exp_12 = []
    input_exp_10 = []
    output_exp_10 = []
    input_exp_8 = []
    output_exp_8 = []
    
    for row in CSV.File(datadir("experiment/SBA15_interp_12.csv"); delim = ",")
        push!(input_exp_12, row.I)
        push!(output_exp_12, row.O)
    end
    
    for row in CSV.File(datadir("experiment/SBA15_interp_10.csv"); delim = ",")
        push!(input_exp_10, row.I)
        push!(output_exp_10, row.O)
    end
    
    for row in CSV.File(datadir("experiment/SBA15_interp_8.csv"); delim = ",")
        push!(input_exp_8, row.I)
        push!(output_exp_8, row.O)
    end
    
    n = 2
    nels = (5n, convert(Int,2.5n),n)
    S = Vec((0.0,0.0,0.0))
    E = Vec((1.0,0.5,0.2))
    grid = generate_grid(Tetrahedron, nels, S, E)
    faces = union(getfaceset(grid,"top"),getfaceset(grid,"bottom"),getfaceset(grid,"left"),getfaceset(grid,"right"),
                   getfaceset(grid,"front"),getfaceset(grid,"back"))
    addfaceset!(grid, "1", faces)

    #Step 2 change the model evaluation
    while isempty(es.stop()) 
        solutions = es.ask() # Returns array of array where inner array holds set of parameters length outer array = N
        #Check the syntax here
        F = Array{Future}(undef, N*3)
        errors = zeros(N*3)
        
        #Complete the for loop
        s = 1
        for idx in 1:length(errors)
            if idx % 3 == 1
                F[idx] = @spawnat idx+1 (eval_model(solutions[s], w[3], input_exp_12, output_exp_12, grid))
            elseif idx % 3 == 2
                F[idx] = @spawnat idx+1 (eval_model(solutions[s], w[2], input_exp_10, output_exp_10,grid))
            elseif idx % 3 == 0
                F[idx] = @spawnat idx+1 (eval_model(solutions[s], w[1], input_exp_8, output_exp_8,grid))
                s += 1
            end
        end
       
        
        # fetch the errors entry 
        for i in 1:N*3
            errors[i] = fetch(F[i])
        end
                
        # Reduce the N*3 length array errors to an array of length N (adding up errors)
        errors_red = sum.(Iterators.partition(errors,3))
                 
        es.tell(solutions,errors_red) 
        #This is only for saving/logging/printing current status
        es.logger.add() 
        es.disp() 
    end
return errors_red
end

calibrate (generic function with 1 method)

In [ ]:
@time calibrate(cma, N)

(2_w,5mirr1)-aCMA-ES (mu_w=1.6,w_1=73%) in dimension 5 (seed=682031, Tue Aug  4 17:03:54 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1      5 1.108097888615528e+03 1.0e+00 4.95e-01  5e-01  5e-01 17:56.5
    2     10 1.062308207765336e+03 1.2e+00 4.63e-01  4e-01  5e-01 36:46.8
    3     15 1.058122516027674e+03 1.2e+00 4.85e-01  5e-01  5e-01 56:23.2
    4     20 1.012190630768107e+03 1.3e+00 5.51e-01  5e-01  6e-01 74:59.0
    5     25 9.901679179524349e+02 1.5e+00 7.02e-01  6e-01  9e-01 94:41.3
    6     30 8.736061682325211e+02 1.8e+00 9.28e-01  9e-01  1e+00 115:08.2
    7     35 8.141849296482072e+02 2.0e+00 1.17e+00  1e+00  2e+00 133:41.0
    8     40 4.599602443456180e+02 2.2e+00 1.68e+00  2e+00  2e+00 155:04.2
    9     45 6.165392410459820e+02 2.3e+00 1.88e+00  2e+00  2e+00 170:09.7
   10     50 6.900371553295228e+01 2.1e+00 2.81e+00  3e+00  4e+00 187:11.8
